# Fast Gradient Sign Attack on some ImageNet samples on a Trained ResNet-18

In [1]:
import torch
import torch.nn as nn
from torchvision import models
from torchvision import transforms

from src.data.ImageNet300 import ImageNet300Dataset
from src.utils.getimagenetclasses import get_classes
from src.attacks.attacks import FastGradientSign

In [2]:
resnet_pretrained = models.resnet18(pretrained = True)
resnet_pretrained.eval()
pass

root_dir = r"C:\Users\willi\Documents\in5400\mand1\prelimcode\students\data\imagenetval300imgs\imagenet300"
xmllabeldir = r"C:\Users\willi\Documents\in5400\mand1\prelimcode\students\data\imagenetval300imgs\val"
synsetfile = r"C:\Users\willi\Documents\in5400\mand1\prelimcode\students\synset_words.txt"

image_transforms =  transforms.Compose([
          transforms.Resize((256,256)),
          transforms.ToTensor(),
          transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])


dataset = ImageNet300Dataset(root_dir, xmllabeldir, synsetfile, 300, image_transforms)
dataloader = torch.utils.data.DataLoader(dataset, batch_size = 8, shuffle = False)

classes = get_classes()

In [3]:
for samp in dataloader:
    break
loss_fn = nn.CrossEntropyLoss()
fgs = FastGradientSign(resnet_pretrained, loss_fn, return_logits=True)

## Attack


In [4]:
org_pred, new_pred, outputs, new_outputs = fgs(samp['image'], samp['label'])

## How many miss classified


In [5]:
missclassified = (org_pred != new_pred)
torch.sum(missclassified).item(), missclassified

(5, tensor([False, False,  True,  True,  True,  True,  True, False]))

## Choose a sample

In [6]:
samp_idx = 4

## Logits of most confident class:


In [7]:
nn.functional.softmax(outputs, dim = 1)[samp_idx].max(), nn.functional.softmax(outputs, dim = 1)[samp_idx].argmax().item()

(tensor(0.2674, grad_fn=<MaxBackward1>), 520)

## Logits of most confident class AFTER ATTACK


In [8]:
nn.functional.softmax(new_outputs, dim = 1)[samp_idx].max(), nn.functional.softmax(new_outputs, dim = 1)[samp_idx].argmax().item()

(tensor(0.0632, grad_fn=<MaxBackward1>), 452)

## Logits of the old predicted class AFTER ATTACK

In [9]:
nn.functional.softmax(new_outputs, dim = 1)[samp_idx][org_pred[samp_idx]]

tensor(0.0036, grad_fn=<SelectBackward>)